#**Maestría en Inteligencia Artificial Aplicada**
##**Curso: Inteligencia Artificial y Aprendizaje Automático**
###Tecnológico de Monterrey
###Prof Luis Eduardo Falcón Morales

## **Adtividad de la Semana 10**
###**Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Nombres y matrículas de los integrantes del equipo:**

*   Elber Aguilar Pérez - A01793568
*   José Francisco Muñoz Del Angel - A01794174
*   Daniel Roberto Meneses Leon - A01794274
*   
*   


En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [100]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import TruncatedSVD

#**Ejercicio-1.** 

In [101]:
ratings_url="https://raw.githubusercontent.com/PosgradoMNA/IAAEquipo59/main/rating_final.csv?token=GHSAT0AAAAAABW3GV3I5XETEVXZID5CG3D2Y34FGOQ"
geoplaces_url="https://raw.githubusercontent.com/PosgradoMNA/IAAEquipo59/main/geoplaces2.csv?token=GHSAT0AAAAAABW3GV3I5XETEVXZID5CG3D2Y34FGOQ"
df_ratings = pd.read_csv(ratings_url)
df_geoplaces = pd.read_csv(geoplaces_url, encoding="latin-1")
print('ratings: ', df_ratings.shape)
df_ratings.head()


ratings:  (1161, 5)


,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [102]:
print('geoplaces: ', df_geoplaces.shape)
df_geoplaces.head()

geoplaces:  (130, 21)


,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none


In [103]:
# merge df_geoplaces into df_ratings using the placeID in order to obtain the name
df = pd.DataFrame()
df['users'] = df_ratings[['userID']]
df['place_id'] = df_ratings[['placeID']]
df['names'] = df_geoplaces.merge(df_ratings, on='placeID')[['name']]
df['food_rating'] = df_ratings[['food_rating']]
print(df.shape)

df.head()

(1161, 4)


,users,place_id,names,food_rating
0,U1077,135085,Kiku Cuernavaca,2
1,U1077,135038,Kiku Cuernavaca,2
2,U1077,132825,Kiku Cuernavaca,2
3,U1077,135060,Kiku Cuernavaca,2
4,U1068,135104,Kiku Cuernavaca,1


#**Ejercicio-2.**

In [104]:
# getting utility matrix using SVD 
UtMx = df.pivot_table(index='users', columns='names', values='food_rating', fill_value=0)
UtMx.head(10)

names,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
users,,,,,,,,,,,,,,,,,,,,,
U1001,0.0,0,0.0,0.0,0.0,0.000000,0.0,0,0.0,0,...,0.0,0,0.0,0.0,0.0,0,0,0.0,0,0.0
U1002,0.0,0,0.0,0.0,0.0,1.571429,0.0,0,0.0,0,...,0.0,0,0.0,0.0,0.0,0,0,0.0,0,0.0
U1003,0.0,0,0.0,1.5,0.0,0.000000,0.0,0,0.0,0,...,0.0,0,0.0,0.0,0.0,0,0,2.0,0,0.0
U1004,0.0,0,0.0,0.0,0.0,0.000000,0.0,0,0.0,0,...,0.0,0,0.0,0.0,0.0,0,0,0.0,0,0.0
U1005,0.0,0,0.0,0.0,0.0,0.000000,0.0,0,0.0,0,...,0.0,0,0.0,0.0,0.0,0,0,0.0,0,0.0
U1006,0.0,0,1.0,0.0,0.0,0.000000,0.0,0,0.0,0,...,0.0,0,0.0,0.0,0.0,0,0,0.0,0,0.0
U1007,0.0,0,0.0,0.0,0.0,0.000000,0.0,0,0.0,0,...,0.0,0,0.0,0.0,0.0,0,0,0.0,0,0.0
U1008,0.0,0,0.0,0.0,0.0,0.000000,0.0,0,0.0,0,...,0.0,0,0.0,0.0,0.0,0,0,0.0,0,0.0
U1009,0.0,0,0.0,0.0,0.0,0.000000,0.0,0,0.0,0,...,0.0,0,0.0,0.0,0.0,0,0,0.0,0,0.0


In [105]:
UtMx.shape

(138, 129)

In [106]:
X=UtMx.T
X.shape

(129, 138)

In [107]:
# SVD
SVD = TruncatedSVD(n_components=137, random_state=7)
result = SVD.fit_transform(X)
result.shape

(129, 129)

In [108]:
# saving results into a dataframe
componentes = pd.DataFrame(result, index=X.index)
componentes.head()

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
names,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,1.422166e-16,-1.766348e-16,-3.731299e-16,-3.050232e-16,5.162869e-16,5.920111e-17,-2.162009e-16,-8.672837e-17,2.706566e-16,-7.599888e-16,...,6.938894e-18,-5.551115e-17,-1.387779e-17,1.387779e-17,-2.081668e-17,-6.938894e-18,-1.734723e-17,2.775558e-17,-7.703720e-34,-1.387779e-17
Arrachela Grill,4.356184e-03,-1.962324e-03,3.191575e-02,2.955450e-02,-1.911935e-05,4.975730e-05,3.105977e-02,3.059644e-03,-1.932894e-03,1.617761e-02,...,-7.731076e-25,1.796079e-23,-6.135986e-24,-3.296601e-24,1.315068e-23,6.339124e-24,-1.086095e-23,3.508661e-24,6.719869e-30,-5.425870e-17
Cabana Huasteca,1.469613e-02,-3.687838e-03,-5.336589e-03,-2.183872e-02,1.527005e-05,2.969554e-03,-4.713000e-04,3.757643e-04,-3.258289e-03,3.238584e-02,...,-1.214306e-17,6.938894e-18,-6.938894e-18,3.469447e-18,4.163336e-17,-6.938894e-18,-6.938894e-18,4.336809e-19,1.473375e-29,1.491862e-16
Cafe Chaires,7.169961e-01,-3.313638e-01,6.586748e-01,1.122478e-01,-5.173801e-05,1.355701e-02,-7.666534e-03,-1.484314e-02,-1.020942e-02,9.232547e-02,...,6.457880e-24,-4.389001e-24,-2.943026e-24,3.823783e-24,-1.354949e-23,1.347719e-23,-1.309464e-23,-2.694636e-25,3.112800e-29,-4.978104e-16
Cafeteria cenidet,1.274304e-01,2.225535e-02,3.397300e-01,2.149691e-01,-1.333624e-04,-1.081254e-02,-2.538654e-03,-3.687456e-03,6.549870e-03,-7.005429e-02,...,-6.099326e-24,1.126350e-23,-4.293044e-25,-4.693260e-24,1.924751e-23,-5.596679e-24,3.098843e-24,2.224243e-24,-2.718071e-30,8.561965e-17


In [109]:
# Valores singulares
print(result)

[[ 1.42216636e-16 -1.76634823e-16 -3.73129875e-16 ...  2.77555756e-17
  -7.70371978e-34 -1.38777878e-17]
 [ 4.35618360e-03 -1.96232427e-03  3.19157455e-02 ...  3.50866146e-24
   6.71986907e-30 -5.42586968e-17]
 [ 1.46961345e-02 -3.68783757e-03 -5.33658867e-03 ...  4.33680869e-19
   1.47337493e-29  1.49186219e-16]
 ...
 [ 1.04597649e+00 -4.89791413e-01  1.02941383e+00 ...  1.28284879e-24
  -1.98011170e-29  1.76885235e-16]
 [ 3.00771079e-02 -9.04745450e-03 -1.66968380e-01 ... -2.05167992e-25
   4.15476782e-29 -2.07673678e-16]
 [ 8.82267436e-03 -3.98066591e-03  6.51438469e-02 ...  1.49152177e-24
  -1.66193309e-29 -2.04201915e-17]]


#**Ejercicio-3.**

In [110]:
# Pearon correlation
corr = np.corrcoef(result)
corr.shape


/Users/dmeneses/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/dmeneses/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


(129, 129)

In [111]:
# getting the top 10 best recommendations for subway place
names_restaurants = UtMx.columns
names_restaurants_list = list(names_restaurants)
subway = names_restaurants_list.index('Subway')
subway

88

Obteniendo los valores de  correlacion de Subway y los 10 restaurantes mas relacionados.

In [112]:
# getting the correlation values for subway
corr_subway = corr[subway]
corr_subway.shape

# getting the correlation values for subway and filtering nan values
# corr_subway = corr_subway[~np.isnan(corr_subway)]
corr_list = list(names_restaurants[(corr_subway<1.0)])
corr_list_values=list()
# printing the top 10 restaurants correlation values for subway and the other restaurants
for i in corr_subway.argsort()[::-1][:11]:
    if i == subway:
        continue
    print('  %s' % names_restaurants_list[i], ' - ', corr_subway[i])



  la Cochinita Pibil Restaurante Yucateco  -  nan
  Restaurante la Cantina  -  nan
  rockabilly   -  nan
  La Cantina Restaurante  -  0.42999993411230725
  Carls Jr  -  0.27228599915440604
  Hamburguesas saul  -  0.26139034949621465
  Restaurante Versalles  -  0.17637810328113696
  la parroquia  -  0.13687898844240518
  Luna Cafe  -  0.13511715765926172
  Hamburguesas La perica  -  0.0110295872177046


#**Ejercicio-4.**

In [113]:
None

#**Ejercicio-5.**

In [114]:
None

#**Ejercicio-6.**

In [115]:
None

#**Ejercicio-7.**

In [116]:
None

#**Ejercicio-8.**

In [117]:
None

###**Fin de la Actividad de la semana 10 - Sistemas de Recomendación y Reducción de Dimensionalidad con SVD.**